You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from two specific endpoints: 
	All launches: https://api.spacexdata.com/v3/launches
	All rockets: https://api.spacexdata.com/v3/rockets
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Merge Operation:
o	Perform a "merge" (or join) operation between the launches data and the rockets data.
o	Merge Logic: For each launch record, add the rocket's name (rocket_name) from the rockets dataset. The match should be based on the rocket.rocket_id field present in each launch record corresponding to the rocket_id field in each rocket record.
o	The final result should be a list of dictionaries, where each dictionary represents a launch enriched with its corresponding rocket name.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URLs, making them easily modifiable.
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, number of rockets retrieved, starting merge operation, number of records after merge, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + merge). Print this time after the merge operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting list from the merge operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.

In [0]:
import requests
import logging
import time
import json
from datetime import timedelta

# Configuration parameters
LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
HTTPBIN_UPLOAD_URL = "https://httpbin.org/post"

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def fetch_data(url, data_name):
    """Fetch data from API with error handling"""
    try:
        logger.info(f"Starting {data_name} data retrieval from {url}")
        start_time = time.time()
        
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raises exception for 4XX/5XX status codes
        
        data = response.json()
        elapsed = time.time() - start_time
        logger.info(f"Retrieved {len(data)} {data_name} records in {elapsed:.2f} seconds")
        return data
    
    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching {data_name} data: {str(e)}")
        raise

def merge_launches_with_rockets(launches, rockets):
    """Merge launches data with rockets data"""
    logger.info("Starting merge operation")
    start_time = time.time()
    
    # Create a lookup dictionary for rockets
    rocket_lookup = {rocket['rocket_id']: rocket['rocket_name'] for rocket in rockets}
    
    # Enrich each launch with rocket name
    enriched_launches = []
    for launch in launches:
        try:
            rocket_id = launch['rocket']['rocket_id']
            rocket_name = rocket_lookup.get(rocket_id, "Unknown")
            
            # Create a new dictionary to avoid modifying the original
            enriched_launch = launch.copy()
            enriched_launch['rocket_name'] = rocket_name
            enriched_launches.append(enriched_launch)
            
        except KeyError as e:
            logger.warning(f"Skipping launch due to missing key: {str(e)}")
    
    elapsed = time.time() - start_time
    logger.info(f"Merged {len(enriched_launches)} records in {elapsed:.2f} seconds")
    return enriched_launches

def upload_to_httpbin(data):
    """Upload data to httpbin and verify response"""
    logger.info(f"Starting upload to {HTTPBIN_UPLOAD_URL}")
    start_time = time.time()
    
    try:
        json_data = json.dumps(data)
        response = requests.post(HTTPBIN_UPLOAD_URL, data=json_data, headers={'Content-Type': 'application/json'})
        response.raise_for_status()
        
        elapsed = time.time() - start_time
        logger.info(f"Upload successful (status {response.status_code}) in {elapsed:.2f} seconds")
        return True
    
    except requests.exceptions.RequestException as e:
        logger.error(f"Upload failed: {str(e)}")
        return False

def main():
    """Main execution pipeline"""
    pipeline_start_time = time.time()
    
    try:
        # Measure data retrieval and merge time
        operations_start_time = time.time()
        
        # Retrieve data
        launches = fetch_data(LAUNCHES_API_URL, "launches")
        rockets = fetch_data(ROCKETS_API_URL, "rockets")
        
        # Merge data
        merged_data = merge_launches_with_rockets(launches, rockets)
        
        operations_elapsed = time.time() - operations_start_time
        logger.info(f"Data operations completed in {operations_elapsed:.2f} seconds")
        
        # Upload result
        upload_success = upload_to_httpbin(merged_data)
        
        # Calculate total pipeline time
        pipeline_elapsed = time.time() - pipeline_start_time
        logger.info(f"Total pipeline execution time: {timedelta(seconds=pipeline_elapsed)}")
        
        return {
            "status": "success",
            "merged_records": len(merged_data),
            "upload_success": upload_success,
            "operations_time_seconds": operations_elapsed,
            "total_time_seconds": pipeline_elapsed
        }
    
    except Exception as e:
        logger.error(f"Pipeline failed: {str(e)}")
        return {
            "status": "error",
            "error": str(e)
        }

# Execute the pipeline
if __name__ == "__main__":
    result = main()
    display(result)  # For Databricks notebook display